# Least Squares using the SVD

In [4]:
import numpy as np
import numpy.linalg as la
import scipy.linalg as spla

In [12]:
# tall and skinny w/nullspace
np.random.seed(12)
A = np.random.randn(6, 4)
b = np.random.randn(6)
A[3] = A[4] + A[5]
A[1] = A[5] + A[1]
A[2] = A[3] + A[1]
A[0] = A[3] + A[1]

### Part I: Singular least squares using QR

Solve: $Ax= b$

QR factorization of A: $A = QR$ 
where $Q$ is orthogonal ($Q^TQ = I$) and  $R$ upper triagular.

From above properties, we get
	
$Α = QR$ \( \rightarrow \) $Q^TA=Q^TQR $ \( \rightarrow \) $Q^TA = IR$ \( \rightarrow \)$R = Q^TA $

Έχουμε $Αx = b$ \( \rightarrow \) $QRx= b $ \( \rightarrow \) $Q^TQRx= Q^Tb $ \( \rightarrow \) $Rx= Q^Tb $

$\mathbf{Q}^{\mathbf{T}} \mathbf{A}=\mathbf{Q}^{\mathbf{T}} \mathbf{Q} \mathbf{R}=\mathbf{D} \mathbf{R}$

Let's see how successfully we can solve the least squares problem **when the matrix has a nullspace** using QR.

**The Nullspace of a Matrix**

The solution sets of homogeneous linear systems provide an important source of vector spaces. Let $A$ be an m by n matrix, and consider the homogeneous system  $A \mathbf{x}=\mathbf{0}$. Since $A$ is m by n, the set of all vectors x which satisfy this equation forms a subset of $R^n$ . (This subset is nonempty, since it clearly contains the zero vector: $x = 0$ always satisfies $A x = 0$.) This subset actually forms a subspace of $R^n$ , called the nullspace of the matrix $A$ and denoted $N(A)$. To prove that $N(A)$ is a subspace of $R^n$, closure under both addition and scalar multiplication must be established.


In [14]:
Q, R = la.qr(A)

In [17]:
R.round(3)

array([[-4.526,  3.492, -0.204, -3.647],
       [ 0.   ,  0.796,  0.034,  0.603],
       [ 0.   ,  0.   , -1.459,  0.674],
       [ 0.   ,  0.   ,  0.   ,  0.   ]])

We can choose `x_qr[3]` as we please:

In [42]:
x_qr = np.zeros(A.shape[1])

In [55]:
x_qr[3] = 5
print(Q,"\n", Q[:3],"\n", Q[:3,3])

[[-0.61188743 -0.04283489 -0.09151134  0.77169406]
 [-0.27729009 -0.53684417 -0.41637733 -0.29285956]
 [-0.61188743 -0.04283489 -0.09151134 -0.4788345 ]
 [-0.33459734  0.49400928  0.32486599 -0.29894001]
 [-0.22371086 -0.16655366  0.73321071  0.00608045]
 [-0.11088649  0.66056294 -0.40834472  0.00608045]] 
 [[-0.61188743 -0.04283489 -0.09151134  0.77169406]
 [-0.27729009 -0.53684417 -0.41637733 -0.29285956]
 [-0.61188743 -0.04283489 -0.09151134 -0.4788345 ]] 
 [ 0.77169406 -0.29285956 -0.4788345 ]


In [32]:
QTbnew = Q.T.dot(b)[:3,] - R[:3, 3] * x_qr[3]
x_qr[:3] = spla.solve_triangular(R[:3,:3], QTbnew, lower=False)

Let's take a look at the residual norm and the norm of `x_qr`:

In [33]:
R.dot(x_qr)-Q.T.dot(b)[:4]

array([ 0.00000000e+00,  0.00000000e+00, -1.11022302e-16, -1.07977253e+00])

In [34]:
la.norm(A.dot(x_qr)-b, 2)

2.126715288803098

In [35]:
la.norm(x_qr, 2)

0.8239351297413158

Choose a different `x_qr[3]` and compare residual and norm of `x_qr`.

residual and norm have larger values for a random `x_qr[3]`

--------------
### Part II: Solving least squares using the SVD
Now compute the SVD of $A$:

In [25]:
U, sigma, VT = la.svd(A)

Make a matrix `Sigma` of the correct size:

In [26]:
Sigma = np.zeros(A.shape)
Sigma[:4,:4] = np.diag(sigma)

And check that we've actually factorized `A`:

In [27]:
(U.dot(Sigma).dot(VT) - A).round(4)

array([[ 0., -0.,  0.,  0.],
       [ 0., -0.,  0.,  0.],
       [ 0., -0.,  0.,  0.],
       [ 0., -0., -0.,  0.],
       [ 0., -0., -0.,  0.],
       [ 0.,  0.,  0.,  0.]])

Now define `Sigma_pinv` as the "pseudo-"inverse of `Sigma`, where "pseudo" means "don't divide by zero":

In [28]:
Sigma_pinv = np.zeros(A.shape).T
Sigma_pinv[:3,:3] = np.diag(1/sigma[:3])
Sigma_pinv.round(3)

array([[ 0.147,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.624,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  1.055,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

Now compute the SVD-based solution for the least-squares problem:

In [29]:
x_svd = VT.T.dot(Sigma_pinv).dot(U.T).dot(b)

In [30]:
la.norm(A.dot(x_svd)-b, 2)

2.1267152888030982

In [31]:
la.norm(x_svd)

0.77354943014895838

* What do you observe about $\|\text{x_svd}\|_2$ compared to $\|\text{x_qr}\|_2$?
* Is $\|\text{x_svd}\|_2$ compared to $\|\text{x_qr}\|_2$?

 $\|\text{x_svd}\|_2 \space $ is smaller than $\space \|\text{x_qr}\|_2$